# Daily data download and first processing

This notebook is part of a toolset to analyse and visualise data on the COVID-19 epidimic and will be run once a day.<br>
Though this can only be done locally on my machine, the data will be saved in ../data/ in this repository.

We will download the ECDC's daily datasheet (https://ourworldindata.org/coronavirus-source-data), containing amount of new cases and deaths due to the COVID-19 outbreak per country per day as an excel sheet.<br>
After which we will slip it into three datasets: daily cases per country, daily deaths per country, and a set containing the populations of the countries in 2018.

I will comment allong the way on what is happening and regularly print what we're working with for transparency.

## Downloading

First we'll have to download the ECDC datasheet.

In [1]:
import urllib.request               # Calls url to downlload daily file
import datetime                     # Provides current date

try:
    today = datetime.date.today() - datetime.timedelta(days=1)
    url = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-' + str(today) + '.xlsx'

    # Second argument in urlretrieve is the destination and filename for saving
    _ = urllib.request.urlretrieve(url, '../data/ECDC_data.xlsx')
    
except: 
    print("No new data.")

Next we'll need to open the downloaded .xlsx file as a pandas dataframe so we can easelly manipulate the data.
Pandas is a powerfull python library to create and manipulate dataframes that is used throughout this project.

We won't go into detail how to use this library, since that would be to expansive for this project. Though it is very 'googleable' and the documenation is expansive.

In [2]:
import pandas as pd                   # Tool to manipulate datasets

df = pd.read_excel('../data/ECDC_data.xlsx')
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-04-07,7,4,2020,38,0,Afghanistan,AF,AFG,37172386.0
1,2020-04-06,6,4,2020,29,2,Afghanistan,AF,AFG,37172386.0
2,2020-04-05,5,4,2020,35,1,Afghanistan,AF,AFG,37172386.0
3,2020-04-04,4,4,2020,0,0,Afghanistan,AF,AFG,37172386.0
4,2020-04-03,3,4,2020,43,0,Afghanistan,AF,AFG,37172386.0
...,...,...,...,...,...,...,...,...,...,...
9305,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
9306,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
9307,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
9308,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


## Processing

Now that we have the data in a form in which we can mold it, we can start working.<br>
As a first step we will clean out the table a bit since we have quite some values that we do not need.

Considering the 'DateRep' column contains a datetime that Python can translate we will delete the unneeded time attributes, as well as the geoid's since I will not be needing them yet.
This can be changed at a later time in case we want to combine other datasets containing country data.

In [3]:
df = df.drop(columns=['day', 'month', 'year', 'geoId', 'countryterritoryCode'])
display(df)

,dateRep,cases,deaths,countriesAndTerritories,popData2018
0,2020-04-07,38,0,Afghanistan,37172386.0
1,2020-04-06,29,2,Afghanistan,37172386.0
2,2020-04-05,35,1,Afghanistan,37172386.0
3,2020-04-04,0,0,Afghanistan,37172386.0
4,2020-04-03,43,0,Afghanistan,37172386.0
...,...,...,...,...,...
9305,2020-03-25,0,0,Zimbabwe,14439018.0
9306,2020-03-24,0,1,Zimbabwe,14439018.0
9307,2020-03-23,0,0,Zimbabwe,14439018.0
9308,2020-03-22,1,0,Zimbabwe,14439018.0


To make our next step a little easier we will create two lists. One containing every date and another containing every country that can be found in the database. Both will be ordered so we can fill in all the datasets in the same alphabetical/accending order.<br>
We will also exchange the _'s in the country name for a whitespace, since this looks better.

In [4]:
dates_all = []
countries = []

for date in df['dateRep']:
    if date not in dates_all:
        dates_all.append(date)

for country in df['countriesAndTerritories']:
    if country not in countries:
        countries.append(country)
        
dates_all.sort()
countries.sort()

countries_all = [country.replace('_', ' ') for country in countries]

In [5]:
africa = ['Zimbabwe', 'South Sudan', 'Malawi', 'Niger', 'Nigeria', 'Zambia','United Republic of Tanzania','Uganda','Tunisia','Togo','Sudan','South Africa','Somalia','Sierra Leone','Seychelles', 'Senegal','Rwanda', 'Namibia','Mozambique','Morocco','Mauritius','Mauritania','Mali','Madagascar','Libya','Liberia', 'Kenya','Guinea Bissau','Guinea', 'Ghana', 'Gambia', 'Gabon','Ethiopia', 'Eswatini','Eritrea','Equatorial Guinea','Egypt', 'Djibouti', 'Democratic Republic of the Congo','Cote dIvoire', 'Congo', 'Chad', 'Central African Republic', 'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde']
asia = ['Cases on an international conveyance Japan', 'Vietnam','Uzbekistan', 'United Arab Emirates','Turkey','Timor Leste','Thailand','Taiwan','Syria','Sri Lanka','South Korea','Singapore','Saudi Arabia','Qatar', 'Philippines','Palestine','Pakistan','Oman', 'Nepal','Myanmar', 'Mongolia','Maldives','Malaysia','Lebanon','Laos','Kyrgyzstan', 'Kuwait','Jordan', 'Japan', 'Israel','Iraq','Iran','Indonesia','India','China', 'Afghanistan', 'Armenia', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei Darussalam', 'Cambodia']
europe = ['Czechia', 'United Kingdom','Ukraine','Switzerland','Sweden','Spain','Slovenia','Slovakia','Serbia','San Marino','Romania', 'Russia','Portugal','Poland','Norway', 'North Macedonia','Netherlands','Monaco','Montenegro','Moldova','Malta','Luxembourg', 'Lithuania','Liechtenstein','Latvia','Kosovo','Kazakhstan', 'Jersey','Italy', 'Isle of Man','Ireland','Azerbaijan','Georgia','Iceland', 'Hungary', 'Holy See','Guernsey', 'Greece', 'Gibraltar', 'Germany', 'France', 'Finland', 'Faroe Islands', 'Estonia','Denmark', 'Cyprus', 'Croatia', 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria']
north_america = ['Belize', 'United States of America','United States Virgin Islands','Turks and Caicos islands','Sint Maarten','Saint Vincent and the Grenadines','Saint Lucia','Saint Kitts and Nevis','Puerto Rico', 'Panama','Nicaragua', 'Montserrat','Mexico', 'Jamaica', 'Honduras', 'Haiti', 'Guatemala', 'Grenada', 'Greenland','El Salvador','Dominican Republic', 'Dominica', 'Cuba', 'Cayman Islands', 'Anguilla', 'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Bermuda', 'British Virgin Islands', 'Canada']
oceania = ['Papua New Guinea','Northern Mariana Islands','New Zealand','New Caledonia','Guam','French Polynesia', 'Fiji','Australia']
south_america = ['Falkland Islands (Malvinas)', 'Bonaire, Saint Eustatius and Saba', 'Venezuela','Uruguay','Trinidad and Tobago','Suriname','Peru','Paraguay', 'Guyana', 'Ecuador', 'CuraÃ§ao', 'Costa Rica', 'Colombia', 'Chile', 'Argentina', 'Aruba', 'Bolivia', 'Brazil']

africa.sort()
asia.sort()
europe.sort()
north_america.sort()
south_america.sort()
oceania.sort()

continents = {'Africa': africa, 'Asia': asia, 'Europe': europe, 'North America': north_america, 'Oceania': oceania, 'South America': south_america}

missing = []
for country in countries_all:
    if not any([country in continents[entry] for entry in continents]):
        missing.append(country)
            
to_much = []
for entry in continents:
    for country in continents[entry]:
        if not country in countries_all:
            to_much.append(country)
            
if missing != [] or to_much != []:
    print(f'The follwoing countries are missing: {missing}')
    print(f'The follwoing countries are to much: {to_much}')

Now for the actual data extraction. 

When we create datasets with new dataframe (aka new column or row structure), we will do this by first creating a dictionary. This will allow us to, in a structured way, collect and order the data by making lists, which we can then fold into dataframe.<br>
This dictionaries will be filled with the names of the columns, coupled to the data of that column in list form.

We will use the dates as an index, going from the earliest date to the latest. This means for each country we will need to create two list's, in that same order, containing cases and deaths per day. To be able to add these lists, with country name as key, into their respective dictionaries.

In [6]:
dict_cases = {'Date': dates_all}
dict_deaths = {'Date': dates_all}

# here we need a numerical for loop, instead of loping through the countries list
# since we need to place the countries_final name in the dictionary instead of the countries one
for i in range(len(countries)):
    cases = []
    deaths = []

    country_df = df.loc[df['countriesAndTerritories']==countries[i]]
    collected_dates = country_df['dateRep'].tolist()

    for date in dates_all:
        if date in collected_dates:
            entry = country_df.loc[country_df['dateRep']==date]
            cases.append(entry['cases'].values[0])
            deaths.append(entry['deaths'].values[0])

        else:
            cases.append(0)
            deaths.append(0)

    dict_cases[countries_all[i]] = cases
    dict_deaths[countries_all[i]] = deaths

df_cases_daily = pd.DataFrame(dict_cases).set_index('Date')
df_cases_daily = df_cases_daily.fillna(0)

df_deaths_daily = pd.DataFrame(dict_deaths).set_index('Date')
df_deaths_daily = df_deaths_daily.fillna(0)

In [7]:
df_cases_daily['Global'] = df_cases_daily.sum(axis=1).values
df_deaths_daily['Global']  = df_deaths_daily.sum(axis=1).values

for entry in continents:
    df_cases_daily[entry] = df_cases_daily[continents[entry]].sum(axis=1).values
    df_deaths_daily[entry]  = df_deaths_daily[continents[entry]].sum(axis=1).values

New we're finally ready to create the dataframes for both daily cases and daily deaths. 

We will start with the daily cases. While creating this, for easy visualization, we will place the date as index and fill all the empty cells, when there was no data provided, with 0.

In [8]:
cols = df_cases_daily.columns.tolist()
cols = cols[-7:] + cols[:-7]

In [9]:
df_cases_daily = df_cases_daily[cols]
display(df_cases_daily)

assert((df_cases_daily.iloc[:,:1].values - df_cases_daily.iloc[:,1:7].values.sum(axis=1)).sum() == 0)

,Global,Africa,Asia,Europe,North America,Oceania,South America,Afghanistan,Albania,Algeria,...,United Kingdom,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,27,0,27,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,17,0,17,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-03,77337,496,8791,34788,30836,304,2122,43,18,0,...,4244,1,3,28819,31,3,1,4,3,0
2020-04-04,81884,579,6157,38106,34476,390,2176,0,27,139,...,4450,0,7,32425,17,37,0,0,0,1
2020-04-05,86707,1166,12244,34624,36195,194,2284,35,29,314,...,3735,0,0,34272,14,39,0,1,0,0


For the daily deaths we will do the same thing.

In [10]:
df_deaths_daily = df_deaths_daily[cols]
display(df_deaths_daily)

assert((df_deaths_daily.iloc[:,:1].values - df_deaths_daily.iloc[:,1:7].values.sum(axis=1)).sum() == 0)

,Global,Africa,Asia,Europe,North America,Oceania,South America,Afghanistan,Albania,Algeria,...,United Kingdom,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-03,4672,27,294,3317,955,2,77,0,1,0,...,389,0,0,915,2,0,0,0,1,0
2020-04-04,6604,32,150,5114,1194,8,106,0,1,25,...,684,0,0,1104,0,0,0,0,0,0
2020-04-05,6115,96,444,4005,1433,4,133,1,2,47,...,708,0,0,1344,1,0,2,0,0,0


With that done, the only thing left to do is to get the populations out of ECDC datasheet. Originally this was done at the same time as the previous data extraction, since it uses the same loop though it seemed more transparent to split it up.

Again we will make a dictionary first. Though this time we will make the countries the index, saving us the trouble of having to deal with the countries_final name in the loop itself.

In [11]:
dict_populations = {'Ind': 'Populations'}

for i in range(len(countries)):
    country_pop = df.loc[df['countriesAndTerritories']==countries[i]]['popData2018'].values[-1]
    dict_populations[countries_all[i]] = [country_pop]

df_populations = pd.DataFrame(dict_populations).set_index('Ind')
df_populations = df_populations.fillna(0)

In [12]:
df_populations['Global'] = df_populations.sum(axis=1).values
countries_all.append('Global')

for entry in continents:
    country_mask = [country in continents[entry] for country in countries_all]
    df_populations[entry] = df_populations.loc['Populations', country_mask].sum()
    countries_all.append(entry)

Again we will use this dictionary to create the dataframe.

In [13]:
df_populations = df_populations[cols]
display(df_populations)

assert((df_populations.iloc[:,:1].values - df_populations.iloc[:,1:7].values.sum(axis=1)).sum() == 0)

,Global,Africa,Asia,Europe,North America,Oceania,South America,Afghanistan,Albania,Algeria,...,United Kingdom,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Ind,,,,,,,,,,,,,,,,,,,,,
Populations,7.498732e+09,1.267817e+09,4.415994e+09,769101282.0,575613071.0,40152057.0,430053988.0,37172386.0,2866376.0,42228429.0,...,66488991.0,56318348.0,106977.0,327167434.0,3449299.0,32955400.0,28870195.0,95540395.0,17351822.0,14439018.0


## Saving

With all the datasets created we can now save them to csv files in ../data/, to be used by the other notebooks for further processing, visualisations and later on to train networks and regression models.

It will always save over the same file so that all other applications can just rerun when the data is updated and the data is available in the ../data/ directory.

In [14]:
df_cases_daily.to_csv('../data/cases_daily.csv')
df_deaths_daily.to_csv('../data/deaths_daily.csv')
df_populations.to_csv('../data/populations_2018.csv')